# ID3 - Algorithm 
In this notebook we want to have a close look at the ID3 Algorithm. To do so we first import all necessary libraries which we need. 

In [1]:
import pandas as pd 
import math
import sys
import numpy as np

Since the ID3 Algorithm describes a Decision-Tree we first want to implement a Node Class. 
The Class Node needs the attributes
 - children
 - value
 - isLeaf
 - pred
The attribute children will be initialized as a List. Obviously every node needs a value. With isLeaf we want to check whether the node is a Leaf or not. Leaf is basically the end nodes of a Tree. Last we give our node an attribute pred where we store the prediction result of the tree.

In [2]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

Now that we have implemented our Class Node, we have to think about the decision rules we want to implement. In the case of the ID3 Algorithm we need two function called `entropy` and `info_gain`. The `entropy` function will be used to calculate the uncertainty of the data set. The `info_gain` will calculate the information gain. The information gain is a measure of the difference in entropy from before to after the Data Set is split on a specific attribute. 

In [3]:
def entropy(examples):
    """
    Compute the entropy of a set of examples
    :param examples: set of examples as pandas set
    :return: entropy as float
    """
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        # compute total number of positive and
        # negative samples
        if row["answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        # entropy is zero if there is no data
        return 0.0
    else:
        p = pos / (pos + neg) # p(+)
        n = neg / (pos + neg) # p(-)
        # entropy is
        # h = -(p(+) * log(p(+) + p(-) * log(p(-)))
        return -(p * math.log(p, 2) + n * math.log(n, 2))

def info_gain(examples, attr):
    """
    Compute the information gain given a set of examples and attributes. This element is
    used to split the tree.
    :param examples: pandas frame of examples
    :param attr:  attributes
    :return: information gain as float
    """
    uniq = np.unique(examples[attr])
    # info gain is G = H(S) - \_sum_a H(S|A=a) for data S and attributes a \in A
    # so we first find the entropy of S
    gain = entropy(examples)
    for u in uniq:
        subdata = examples[examples[attr] == u]
        sub_e = entropy(subdata)
        # and then subtract the conditional entropy H(S|A=a)
        # normalized by N_a/N_total
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
    return gain

Now that we have implemented our `info_gain` function we need a function `find_max_feat`. This function shall iterate over all the attributes our Data Set offers and find the attribute with the highest information gain. 

In [4]:
def find_max_feat(examples, attrs):
    """
    Find feature with maximum information gain
    :param examples: pandas frame of examples
    :param attrs: attributes
    :return: feature with max. info gain as str
    """

    max_gain = -1e-10
    max_feat = ""
    print("Remaining examples: ",examples.shape)
    print("Remaining attributes: ",attrs)
    for feature in attrs:
        gain = info_gain(examples, feature)
        print("feature: ",feature)
        print("gain: ", gain)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    return max_feat, max_gain

At this point we want to implement the ID3 Algorithm. The basic idea of the algorithm is summarized as the following
- Calculate the entropy of every attribute a of the Data set 
- Split the Data Set into subsets using the attribute for which the resulting entropy after splitting the information gain is maximized. 
- Make decision tree node containing the attribute
- Recurse on subsets using the remaining attributes

In [5]:
def ID3(examples, attrs, run=0):
    """
    Recursively construct a decision tree using the ID3 Algorithm
    :param examples: a set of examples
    :param attrs: attributes
    :return: returns the root node of the decision tree
    """
    root = Node()

    max_feat, max_gain = find_max_feat(examples, attrs)
    root.value = max_feat

    print ("Iteration Depth %d: Feature" %run, max_feat, "has highest information gain with approx. %.3f bits" % max_gain)


 # get list of attributes of max_feat
    uniq = np.unique(examples[max_feat])

    for u in uniq:
        # find all samples with this particular attribute of max_feat
        subdata = examples[examples[max_feat] == u]
        # check entropy of subdata
        entropy_sub = entropy(subdata)
        if entropy_sub == 0.0 or len(attrs) == 1:
            # if it's zero, create new node as leaf
            # and set answer accordingly
            newNode = Node()
            pos = 0
            neg = 0
            for _, row in subdata.iterrows():
                # compute total number of positive and
                # negative samples
                if row["answer"] == "yes":
                    pos += 1
                else:
                    neg += 1
            if( pos > neg ) :
                leaf_answer = "yes"
            else :
                leaf_answer = "no"
            #leaf_answer =  np.unique(subdata["answer"])
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = leaf_answer
            root.children.append(newNode)
            print("Entropy is zero. Created leaf node with answer %s for value %s" % (leaf_answer[0], u))
        else:
            # if it's not zero, then we are not done yet
            # create new node, set value and attributes
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            print("Entropy is %.3f. Created inner node for value %s" % (entropy_sub, u))
            # call ID3 recursively
            child = ID3(subdata, new_attrs, run+1)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root


To visualize the tree we need a function `printTree`. Also we have to implement a function `predict`. 

In [6]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

def predict(current_node, x):
    if current_node.isLeaf:
        return current_node.pred
    else:
        for child in current_node.children:
            if x[current_node.value] == child.value:
                if len(child.children):
                    return predict(child.children[0], x)
                else:
                    return child.pred


Now we can finally load in our data and try the algorithm. 

In [7]:
data = pd.read_csv("EnjoySport.csv").applymap(str).applymap(str.strip)
print("Loaded the following data set:")
print(data)

Loaded the following data set:
     sky airtemp humidity    wind water forecast answer
0  sunny    warm   normal  strong  warm     same    yes
1  sunny    warm     high  strong  warm     same    yes
2  rainy    cold     high  strong  warm   change     no
3  sunny    warm     high  strong  cool   change    yes


Now we want to initialize a list of all the attributes, but without the answer attribute. 

In [8]:
features = [feat for feat in data]
features.remove("answer")
print("with the following features:")
print(features)

with the following features:
['sky', 'airtemp', 'humidity', 'wind', 'water', 'forecast']


We can now initialize the ID3 tree and print it. 

In [9]:
root = ID3(data, features)
print("done creating tree!\n\n")
print("Final Tree:\n")
printTree(root)
print("each row is a level of the tree and -> indicate leaf nodes\n")

Remaining examples:  (4, 7)
Remaining attributes:  ['sky', 'airtemp', 'humidity', 'wind', 'water', 'forecast']
feature:  sky
gain:  0.8112781244591328
feature:  airtemp
gain:  0.8112781244591328
feature:  humidity
gain:  0.12255624891826566
feature:  wind
gain:  0.0
feature:  water
gain:  0.12255624891826566
feature:  forecast
gain:  0.31127812445913283
Iteration Depth 0: Feature sky has highest information gain with approx. 0.811 bits
Entropy is zero. Created leaf node with answer n for value rainy
Entropy is zero. Created leaf node with answer y for value sunny
done creating tree!


Final Tree:

sky
	rainy ->  no

	sunny ->  yes

each row is a level of the tree and -> indicate leaf nodes



Last we want to make the prediction and print it. 

In [10]:
# prediction
x = data.iloc[0,:-1]
print("prediction for \n", x.to_string(), " is ", predict(root, x) )

prediction for 
 sky          sunny
airtemp       warm
humidity    normal
wind        strong
water         warm
forecast      same  is  yes
